In [1]:
from langchain_community.llms.ollama import Ollama

QUERY_TEXT = "What are the Business Analyst's responsibilities during CTAC's software maintenance process"
model = Ollama(model="llama3")
response_text = model.invoke(QUERY_TEXT)
print(response_text)

As a Business Analyst, your responsibilities during the software maintenance process for CTAC's (Customer Technical Assistance Centers) software would likely include:

1. **Requirements gathering**: Review and analyze the current software functionality to understand its strengths, weaknesses, and areas that need improvement.
2. **Change request analysis**: Evaluate change requests from stakeholders, assess their impact on the system, and prioritize them based on business value and feasibility.
3. **Gap analysis**: Identify gaps between the current software capabilities and the desired future state, highlighting potential issues or areas for improvement.
4. **Solution definition**: Collaborate with developers to define solutions that meet the change requests and gap analyses, ensuring they align with business objectives and requirements.
5. **Testing and quality assurance**: Work with QA teams to develop test plans, scripts, and scenarios to ensure the software changes meet the requirem

# Setup the Vector Database for RAG

In [2]:
import os
import shutil

from langchain_community.document_loaders.directory import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma

from langchain_community.embeddings.ollama import OllamaEmbeddings


def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings


CHROMA_PATH = "../chroma/"
DATA_PATH = "../data/"


def load_documents():
    document_loader = DirectoryLoader(DATA_PATH, show_progress=True)
    return document_loader.load()


def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)


def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")


def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks


def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

## Ensure that the database is empty before running the demo

In [3]:
clear_database()

## Load Documents from data/ into vector store

In [4]:
# Create or update the database
documents = load_documents()
chunks = split_documents(documents)
add_to_chroma(chunks)

  4%|████████▍                                                                                                                                                                                         | 1/23 [00:01<00:38,  1.76s/it]Error loading file ../data/4.0 Software Delivery - Engineering Process (RDM, PR, VV, TS, PI)/1.Process/~$ftware Maintenance Process.docx


PackageNotFoundError: Package not found at '../data/4.0 Software Delivery - Engineering Process (RDM, PR, VV, TS, PI)/1.Process/~$ftware Maintenance Process.docx'

# Example Query with RAG

In [ ]:
import argparse
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama


# Prepare the DB.
embedding_function = get_embedding_function()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Search the DB.
QUERY_TEXT = "What are the Business Analyst's responsibilities during CTAC's software maintenance process"
results = db.similarity_search_with_score(QUERY_TEXT, k=5)

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=QUERY_TEXT)
# print(prompt)

model = Ollama(model="llama3")
response_text = model.invoke(prompt)

sources = [doc.metadata.get("id", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)
return response_text